To generate template for new proposals.

In [37]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

In [38]:
# Initialize the require parameters.
# file name does not require file extension. File will be saved to Downloads folder.
# system_names cannot exceed the character length of 32 as this is excel's sheet names limit.
# system_names should be put in required order.
file_name = 'J12234 MODEC - EQUINOR BACALHAU FPSO - VSAT'
system_names = ['VSAT SYSTEM for MPOWER', 'TWO YEARS SPARES', 'ENGINEERING SERVICES']
quoted_currency = 'USD'
give_discount = True

In [39]:
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
file_path = Path(downloads_folder, file_name)
# Download template file from the internet and write to local folder
url = "https://filedn.com/liTeg81ShEXugARC7cg981h/Proposal_Template.xlsx"
resp = requests.get(url)

with open("Template.xlsx", 'wb') as fd:
    for chunk in resp.iter_content(chunk_size=8192):
        fd.write(chunk)
system_names = [x.upper() for x in system_names]
# Copy sheet from template to new workbook
new_book = xw.Book()
template = xw.Book("Template.xlsx", password=hide.new)
template.sheets['config'].copy(after=new_book.sheets[0])
new_book.sheets['Sheet1'].delete()
template.sheets['Cover'].copy(after=new_book.sheets['config'])

# Set date in Config
new_book.sheets['config'].range('B32').value = datetime.today().strftime('%Y-%m-%d')

# Set up formula in Cover sheet
new_book.sheets['Cover'].range('D7').formula = '=Config!B26'
new_book.sheets['Cover'].range('C42').formula = '=Config!B21'
new_book.sheets['Cover'].range('C43').formula = '=Config!B23'
new_book.sheets['Cover'].range('C44').formula = '=Config!B24'
new_book.sheets['Cover'].range('C45').formula = '=Config!B29'
new_book.sheets['Cover'].range('C46').formula = '=Config!B30'
new_book.sheets['Cover'].range('C47').formula = '=Config!B32'
new_book.sheets['Cover'].range('D39').formula = '=Config!B13'

for system in system_names[::-1]:
    sheet_name = 'Cover'
    template.sheets['System'].copy(after=new_book.sheets[sheet_name])
    sheet_name = system
    new_book.sheets['System'].name = sheet_name
    # Set formula to reference Config.
    new_book.sheets[sheet_name].range('C1').formula = '=Config!B29'
    new_book.sheets[sheet_name].range('C2').formula = '=Config!B30'
    new_book.sheets[sheet_name].range('C3').formula = '=Config!B32'
    new_book.sheets[sheet_name].range('C4').formula = '=Config!B26'
template.sheets['Summary'].copy(after=new_book.sheets['Cover'])
template.sheets['Technical_Notes'].copy(after=new_book.sheets[-1])
template.sheets['T&C'].copy(after=new_book.sheets[-1])
for sheet in new_book.sheet_names:
    if sheet in ['Summary','Technical_Notes', 'T&C']:
        new_book.sheets[sheet].range('C1').formula = '=Config!B29'
        new_book.sheets[sheet].range('C2').formula = '=Config!B30'
        new_book.sheets[sheet].range('C3').formula = '=Config!B32'
        new_book.sheets[sheet].range('C4').formula = '=Config!B26'
template.close()

In [40]:
# Write necessary formula to excel
for system in system_names:
    sheet = new_book.sheets[system]

    # Set default values
    if system == 'ENGINEERING SERVICES':
        sheet.range('I2').value = 0.3
        sheet.range('K1').value = 0
        sheet.range('K2').value = 0
        sheet.range('K3').value = 0
        sheet.range('K4').value = 0
    # Formula to cells
    last_row = sheet.range('C100000').end('up').row
    sheet.range('N6:N' + str(last_row)).formula = '=IF(K6<>"",K6*(1-M6),"")'
    sheet.range('O6:O' + str(last_row)).formula = '=IF(AND(D6<>"", K6<>"",H6<>"OPTION"),D6*N6,"")'
    # Exchange rates
    sheet.range('Q6:Q' + str(last_row)).formula = '=IF(Config!B12="SGD",IF(J6<>"",VLOOKUP(J6,Config!$A$2:$B$10,2,FALSE),""),IF(J6<>"",VLOOKUP(J6,Config!$A$2:$B$10,2,FALSE)/VLOOKUP(Config!$B$12,Config!$A$2:$B$10,2,FALSE),""))'
    sheet.range('R6:R' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"") ,N6*Q6,"")'
    sheet.range('S6:S' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"",H6<>"OPTION") ,D6*R6,"")'
    sheet.range('T6:T' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""), (R6*(1+$K$1+$K$2+$K$3+$K$4))/(1-0.05),"")'
    # Below formula does not capture all options yet
    sheet.range('U6:U' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),IF(H6="OPTION","",D6*T6),"")'
    sheet.range('V6:V' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$1,"")'
    sheet.range('W6:W' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$2,"")'
    sheet.range('X6:X' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$3,"")'
    sheet.range('Y6:Y' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$4,"")'
    sheet.range('Z6:Z' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),T6-(R6+V6+W6+X6+Y6),"")'
    sheet.range('AA6:AA' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),$I$2,"")'
    sheet.range('AC6:AC' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),CEILING(T6/(1-AA6), 1),"")'
    sheet.range('AD6:AD' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION",H6<>"INCLUDED"),D6*AC6,"")'
    sheet.range('AE6:AE' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),IF(AB6<>"",AB6,AC6),"")'
    sheet.range('AF6:AF' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION", H6<>"INCLUDED"),D6*AE6,"")'
    sheet.range('AG6:AG' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION", H6<>"INCLUDED"),AF6-U6,"")'
    sheet.range('AH6:AH' + str(last_row)).formula = '=IF(AND(AG6<>"",AG6<>0),AG6/AF6,"")'
    sheet.range('F6:F' + str(last_row)).formula = '=IF(AE6<>"", AE6,"")'
    sheet.range('G6:G' + str(last_row)).formula = '=IF(AND(F6<>"", H6<>"OPTION", H6<>"INCLUDED"), D6*F6,"")'
    sheet.range('L6:L' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"",H6<>"OPTION"),D6*K6,"")'
    (new_book.sheets['Config'].range('100:100')).copy(sheet.range(str(last_row+2) + ':' + str(last_row+2)))
    sheet.range('F'+ str(last_row+2)).formula = '="Subtotal (" & Config!B12 & ")"'
    sheet.range('F'+ str(last_row+2)).font.bold = True
    sheet.range('G' + str(last_row+2)).formula = '=SUM(G6:G' + str(last_row+1) + ')'
    sheet.range('G' + str(last_row+2)).font.bold = True
    sheet.range('U' + str(last_row+2)).formula = '=SUM(U6:U' + str(last_row+1) + ')'
    sheet.range('U' + str(last_row+2)).font.bold = True
    sheet.range('AF' + str(last_row+2)).formula = '=SUM(AF6:AF' + str(last_row+1) + ')'
    sheet.range('AF' + str(last_row+2)).font.bold = True
    sheet.range('AG' + str(last_row+2)).formula = '=SUM(AG6:AG' + str(last_row+1) + ')'
    sheet.range('AG' + str(last_row+2)).font.bold = True
    sheet.range('AH' + str(last_row+2)).formula = '=AG' + str(last_row+2) + '/AF' + str(last_row+2)
    sheet.range('AH' + str(last_row+2)).font.bold = True


In [41]:
summary_formula = []
collect = [] # Collect formula to be put in summary page.
for sheet in system_names:
    sheet = new_book.sheets[sheet]
    last_row = sheet.range('G100000').end('up').row
    collect = ["='" + sheet.name + "'!$G$" + str(last_row),
               "='" + sheet.name + "'!$U$" + str(last_row)]
            #    "='" + sheet.name + "'!$AF$" + str(last_row)]
    summary_formula.extend(collect)
    collect = []

count = 1
offset = 20
odered_summary_formula = summary_formula[::-1]
sheet = new_book.sheets['Summary']
for system in system_names:
    sheet.range('B' + str(offset)).value = count
    sheet.range('C' + str(offset)).value = system
    sheet.range('D' + str(offset)).formula = odered_summary_formula.pop()
    sheet.range('H' + str(offset)).formula = odered_summary_formula.pop()
    sheet.range('I' + str(offset)).formula = '=D' + str(offset) + '- H' + str(offset)
    sheet.range('J' + str(offset)).formula = '=IF(I' + str(offset) + '<>0,I' + str(offset) + '/D' + str(offset) + ',""'
    count += 1
    offset += 1

(new_book.sheets['Config'].range('106:106')).copy(sheet.range(str(offset) + ':' + str(offset)))
(new_book.sheets['Config'].range('102:102')).copy(sheet.range(str(offset+1) + ':' + str(offset+1)))
sheet = new_book.sheets['Summary']
sheet.range('C' + str(offset+1)).value = '="TOTAL PROJECT (" & Config!B12 & ")"'
sheet.range('D' + str(offset+1)).formula = '=SUMIF(E20:E' + str(offset) + ',"<>OPTION",D20:D' + str(offset) + ')'
sheet.range('E' + str(offset+1)).formula = '=IF(COUNTIF(E20:E' + str(offset) + ',"OPTION"), "Excluding Option", "")'
sheet.range('H' + str(offset+1)).formula = '=SUMIF(E20:E' + str(offset) + ',"<>OPTION",H20:H' + str(offset) + ')'
sheet.range('I' + str(offset+1)).formula = '=D' + str(offset+1) + '- H' + str(offset+1)
sheet.range('J' + str(offset+1)).formula = '=I' + str(offset+1) + '/D' + str(offset+1)
if give_discount:
    (new_book.sheets['Config'].range('103:103')).copy(sheet.range(str(offset+2) + ':' + str(offset+2)))
    (new_book.sheets['Config'].range('104:104')).copy(sheet.range(str(offset+3) + ':' + str(offset+3)))
    sheet.range('C' + str(offset+3)).formula = '="TOTAL PROJECT PRICE AFTER DISCOUNT (" & Config!B12 & ")"'
    sheet.range('D' + str(offset+3)).formula = '=SUM(D' +str(offset+1) + ':D' + str(offset+2) + ')'
    sheet.range('H' + str(offset+3)).formula = '=$H$' +str(offset+1)
    sheet.range('I' + str(offset+3)).formula = '=D' + str(offset+3) + '- H' + str(offset+3)
    sheet.range('J' + str(offset+3)).formula = '=I' + str(offset+3) + '/D' + str(offset+3)
    sheet.range('C' + str(offset+5)).value = "• All the prices are in " + quoted_currency + " excluding GST."
    sheet.range('C' + str(offset+6)).value = "• Total project price does not include prices for optional items set out in the detailed bill of material."
else:
    sheet.range('C' + str(offset+3)).formula = '="• All the prices are in " & Config!B12 & " excluding GST."'
    sheet.range('C' + str(offset+4)).value = "• Total project price does not include prices for optional items set out in the detailed bill of material."

last_row = sheet.range('C100000').end('up').row
sheet.page_setup.print_area = 'A1:F' + str(last_row)

In [42]:
new_book.save( downloads_folder + '/' + file_name + '.xlsx', password=hide.legacy) 
# new_book.close()